# AI FAQ 챗봇 만들기 with RAG

In [1]:
# 환경 변수 세팅
from dotenv import load_dotenv
load_dotenv()

True

In [27]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.schema import Document
import json
from langchain_core.output_parsers import StrOutputParser

1. 데이터 로드

In [3]:
with open('data/faq_chatbot_data.json', 'r', encoding='utf-8') as f:
    faq_data = json.load(f)

In [4]:
faq_data

[{'question': '반품 정책이 어떻게 되나요?',
  'answer': '제품을 수령한 후 14일 이내에 반품이 가능합니다. 반품 신청은 고객센터에서 도와드립니다.'},
 {'question': '배송은 얼마나 걸리나요?',
  'answer': '일반적으로 2~5일 정도 소요되며, 지역에 따라 다를 수 있습니다.'},
 {'question': '회원가입 없이 주문할 수 있나요?',
  'answer': '네, 비회원 주문이 가능합니다. 다만, 주문 조회 및 혜택을 받기 위해 회원가입을 추천드립니다.'},
 {'question': '포인트 적립은 어떻게 되나요?',
  'answer': '구매 금액의 1%가 적립되며, 특별 프로모션 기간 동안 추가 적립이 가능합니다.'},
 {'question': '해외 배송이 가능한가요?',
  'answer': '일부 국가를 대상으로 해외 배송이 가능하며, 배송비는 지역에 따라 상이할 수 있습니다.'},
 {'question': '제품 보증 기간은 얼마나 되나요?',
  'answer': '제품 보증 기간은 기본적으로 1년이며, 일부 제품은 연장 보증이 가능합니다.'},
 {'question': 'A/S 신청은 어떻게 하나요?',
  'answer': '공식 홈페이지 또는 고객센터를 통해 A/S 신청이 가능합니다.'},
 {'question': '재고가 없는 상품은 언제 다시 입고되나요?',
  'answer': '입고 일정은 상품마다 다르며, 알림 설정을 통해 입고 시 안내받을 수 있습니다.'},
 {'question': '구매 후 영수증을 받을 수 있나요?',
  'answer': '네, 주문 내역에서 전자 영수증을 다운로드할 수 있습니다.'},
 {'question': '다른 사람이 대신 주문을 수령할 수 있나요?',
  'answer': '네, 수령인을 지정할 수 있으며 신분증 확인 후 수령 가능합니다.'},
 {'question': '결제 방법에는 어떤 것이 있나요?',
  'answer': '신용카드, 

2. embedding 모델 생성 + vector store 생성

In [5]:
embedding_model = OpenAIEmbeddings(model='text-embedding-3-small')

In [30]:
documents = []

for item in faq_data:
    question = item['question']
    answer = item['answer']
    
    content = f"Q: {question}\nA: {answer}"
    
    doc = Document(
        page_content=content,
        metadata={'source': 'faq'}
    )
    
    documents.append(doc)
    
vector_store = Chroma.from_documents(documents, embedding_model,persist_directory='chroma_db')

In [ ]:
vector_store = Chroma(
    embedding_function=embedding_model,
    persist_directory='chroma_db'
)

3. llm 설정 (모델 생성)

In [17]:
model = ChatOpenAI(model_name='gpt-4o-mini', temperature=0)

4. 프롬프트 및 체인 설정

In [19]:
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
다음은 자주 묻는 질문과 그에 대한 답변입니다.
이 정보를 바탕으로 사용자의 질문에 가장 관련 있는 답변을 제공하세요.
반드시 context만을 사용하여 답변하새요

context: {context}

질문: {question}
답변: 
"""
)


In [28]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3})
question="배송은 얼마나 걸리나요?"
retrivals = retriever.batch([question])
context_text = "\n".join([doc.page_content for doc in retrivals[0]])

chain = prompt | model | StrOutputParser()

In [31]:
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)

5. 사용

In [29]:
chain.invoke({"context": context_text, "question": question})

'일반적으로 2~5일 정도 소요되며, 지역에 따라 다를 수 있습니다.'

In [33]:
qa_chain.run('현대다운')

'죄송하지만, 제공된 context에는 "현대다운"에 대한 정보가 포함되어 있지 않습니다. 추가적인 정보나 질문을 주시면 더 나은 답변을 드릴 수 있습니다.'